In [ ]:
library(clusterProfiler)
library(igraph)
library(Mus.musculus)
library(stringr)

## Load full graph and associate corresponding gene IDs

In [ ]:
graph = read_graph("OmicsIntegratorRun/recAndRanklForestsColored.gml", format = "gml")

In [ ]:
genes <- select(Mus.musculus, keys=vertex_attr(graph)$name, columns=c("ENTREZID", "TXCHROM"), 
                keytype="SYMBOL")
aliasToSymbol <- select(Mus.musculus, genes$SYMBOL[is.na(genes$ENTREZID)], keytype="ALIAS", col=c("ENTREZID", "TXCHROM", "SYMBOL"), fuzzy=T)

In [ ]:
getGeneID <- function(symbol){
    geneID = genes$ENTREZID[genes$SYMBOL == symbol]
    if (is.na(geneID)){
        geneID = aliasToSymbol$ENTREZID[aliasToSymbol$ALIAS == symbol]
    }
    return(geneID)
}

In [ ]:
orderedGeneIDs <- sapply(vertex_attr(graph)$name, getGeneID)

## Load Infomap clusters

In [ ]:
netMap = scan("net.map", character(), sep = "\n")

In [ ]:
# Extract community of each node
startTab = grep("*Nodes", netMap)
endTab = grep("*Links", netMap)
tab = matrix(unlist(sapply(str_remove_all(netMap[(startTab+1):(endTab-1)], '"'), function(x) str_split(x, " "))), nrow = 3)
nodeModules = data.frame(node = tab[2,], module = sapply(tab[1,], function(x) strsplit(x, ":")[[1]][1]))

## Perform enrichment for each cluster

In [ ]:
moduleKeggEnrichment <- function(modNodes){
    # IDs corresponds to annotation order but are starting from 0 instead of 1
    modIDs = orderedGeneIDs[as.numeric(as.character(modNodes)) + 1]
    return(enrichKEGG(modIDs, 'mmu', 'ncbi-geneid'))
}

In [ ]:
enrichments = aggregate(nodeModules$node, list(nodeModules$module), moduleKeggEnrichment, simplify = F)

In [ ]:
headEnrich = rbind(sapply(1:dim(enrichments)[1], function(y) head(enrichments$x[y][[1]])), Cluster = 1:9)

In [ ]:
headEnrich

In [ ]:
sessionInfo()